# 航空公司客户价值分析

## 实验目的：<br>
借助航空公司客户数据，对客户进行分类。<br>
对不同的客户类别进行特征分析，比较不同类别客户的客户价值。<br>
对不同价值的客户类别提供个性化服务，制定相应的营销策略。<br>

### 读取数据，指定编码为gb18030

In [2]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('../data/air_data.csv', encoding='gb18030')

<img src='../data/data1.png'>

<img src='../data/data2.png'>

### 数据描述性分析

In [6]:
data.describe()

,MEMBER_NO,FFP_TIER,AGE,FLIGHT_COUNT,BP_SUM,EP_SUM_YR_1,EP_SUM_YR_2,SUM_YR_1,SUM_YR_2,SEG_KM_SUM,...,ADD_Point_SUM,Eli_Add_Point_Sum,L1Y_ELi_Add_Points,Points_Sum,L1Y_Points_Sum,Ration_L1Y_Flight_Count,Ration_P1Y_Flight_Count,Ration_P1Y_BPS,Ration_L1Y_BPS,Point_NotFlight
count,62988.000000,62988.000000,62568.000000,62988.000000,62988.000000,62988.0,62988.000000,62437.000000,62850.000000,62988.000000,...,62988.000000,62988.000000,62988.000000,62988.0000,62988.000000,62988.000000,62988.000000,62988.000000,62988.000000,62988.000000
mean,31494.500000,4.102162,42.476346,11.839414,10925.081254,0.0,265.689623,5355.376064,5604.026014,17123.878691,...,1355.006223,1620.695847,1080.378882,12545.7771,6638.739585,0.486419,0.513581,0.522293,0.468422,2.728155
std,18183.213715,0.373856,9.885915,14.049471,16339.486151,0.0,1645.702854,8109.450147,8703.364247,20960.844623,...,7868.477000,8294.398955,5639.857254,20507.8167,12601.819863,0.319105,0.319105,0.339632,0.338956,7.364164
min,1.000000,4.000000,6.000000,2.000000,0.000000,0.0,0.000000,0.000000,0.000000,368.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,15747.750000,4.000000,35.000000,3.000000,2518.000000,0.0,0.000000,1003.000000,780.000000,4747.000000,...,0.000000,0.000000,0.000000,2775.0000,700.000000,0.250000,0.288889,0.258150,0.167954,0.000000
50%,31494.500000,4.000000,41.000000,7.000000,5700.000000,0.0,0.000000,2800.000000,2773.000000,9994.000000,...,0.000000,0.000000,0.000000,6328.5000,2860.500000,0.500000,0.500000,0.514252,0.476747,0.000000
75%,47241.250000,4.000000,48.000000,15.000000,12831.000000,0.0,0.000000,6574.000000,6845.750000,21271.250000,...,0.000000,345.000000,0.000000,14302.5000,7500.000000,0.711111,0.750000,0.815091,0.728375,1.000000
max,62988.000000,6.000000,110.000000,213.000000,505308.000000,0.0,74460.000000,239560.000000,234188.000000,580717.000000,...,984938.000000,984938.000000,728282.000000,985572.0000,728282.000000,1.000000,1.000000,0.999989,0.999993,140.000000


### 数据预处理<br>
#### 1. 去除票价为空的数据


In [7]:
exp1 = data['SUM_YR_1'].notnull()
exp2 = data['SUM_YR_2'].notnull()
exp = exp1 & exp2
data_notnull = data.loc[exp, :]
data_notnull.shape

(62299, 44)

#### 2.只保留票价不为0，平均折扣率不为0，总飞行公里数大于0的记录。

In [10]:
index1 = data_notnull['SUM_YR_1'] != 0
index2 = data_notnull['SUM_YR_2'] != 0
index3 = (data_notnull['avg_discount'] != 0) & (data_notnull['SEG_KM_SUM'] > 0)
data = data_notnull[(index1 | index2) & index3]
data.shape

(62044, 44)

### 构建特征

<img src='../data/1.jpg'>

L: LOAD_TIME  观测窗口的结束时间----FFP_DATE	入会时间<br>
R: LAST_TO_END  最后一次乘机时间至观测窗口结束时长<br>
F: FLIGHT_COUNT 观测窗口内的飞行次数 <br>
M: SEG_KM_SUM 观测窗口的总飞行公里数 <br>
C: avg_discount 平均折扣率<br>


In [13]:
data_selection = data[['LOAD_TIME', 'FFP_DATE', 'LAST_TO_END', 'FLIGHT_COUNT', 'SEG_KM_SUM', 'avg_discount']]
data_selection.head()

,LOAD_TIME,FFP_DATE,LAST_TO_END,FLIGHT_COUNT,SEG_KM_SUM,avg_discount
0,2014/3/31,2006/11/2,1,210,580717,0.961639
1,2014/3/31,2007/2/19,7,140,293678,1.252314
2,2014/3/31,2007/2/1,11,135,283712,1.254676
3,2014/3/31,2008/8/22,97,23,281336,1.090870
4,2014/3/31,2009/4/10,5,152,309928,0.970658


In [14]:
data_selection.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62044 entries, 0 to 62978
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LOAD_TIME     62044 non-null  object 
 1   FFP_DATE      62044 non-null  object 
 2   LAST_TO_END   62044 non-null  int64  
 3   FLIGHT_COUNT  62044 non-null  int64  
 4   SEG_KM_SUM    62044 non-null  int64  
 5   avg_discount  62044 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 3.3+ MB


In [15]:
# 数据类型的处理
data_selection['LOAD_TIME'] = pd.to_datetime(data_selection['LOAD_TIME'])
data_selection['FFP_DATE'] = pd.to_datetime(data_selection['FFP_DATE'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
data_selection.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62044 entries, 0 to 62978
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   LOAD_TIME     62044 non-null  datetime64[ns]
 1   FFP_DATE      62044 non-null  datetime64[ns]
 2   LAST_TO_END   62044 non-null  int64         
 3   FLIGHT_COUNT  62044 non-null  int64         
 4   SEG_KM_SUM    62044 non-null  int64         
 5   avg_discount  62044 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(3)
memory usage: 3.3 MB


In [22]:
# 构建L特征
L = data_selection['LOAD_TIME'] - data_selection['FFP_DATE']
# L
L = L.astype('str').str.split().str[0]
L = L.astype('int') / 30
L = np.round(L, 2)
L

0         90.20
1         86.57
2         87.17
3         68.23
4         60.53
          ...  
62974    108.30
62975     65.37
62976     45.40
62977     15.53
62978     36.07
Length: 62044, dtype: float64

In [23]:
# 合并特征
data_feature = pd.concat(objs=[L, data_selection.iloc[:, 2:]], axis=1)
data_feature.head()

,0,LAST_TO_END,FLIGHT_COUNT,SEG_KM_SUM,avg_discount
0,90.20,1,210,580717,0.961639
1,86.57,7,140,293678,1.252314
2,87.17,11,135,283712,1.254676
3,68.23,97,23,281336,1.090870
4,60.53,5,152,309928,0.970658


In [24]:
data_feature = data_feature.rename(columns={0: 'L'})
data_feature.head()

,L,LAST_TO_END,FLIGHT_COUNT,SEG_KM_SUM,avg_discount
0,90.20,1,210,580717,0.961639
1,86.57,7,140,293678,1.252314
2,87.17,11,135,283712,1.254676
3,68.23,97,23,281336,1.090870
4,60.53,5,152,309928,0.970658


### 数据标准化处理

In [25]:
data_feature_scaled = (data_feature - data_feature.mean()) / data_feature.std()
data_feature_scaled.head()

,L,LAST_TO_END,FLIGHT_COUNT,SEG_KM_SUM,avg_discount
0,1.435706,-0.944948,14.034016,26.761154,1.295540
1,1.307268,-0.911894,9.073213,13.126864,2.868176
2,1.328498,-0.889859,8.718869,12.653481,2.880950
3,0.658357,-0.416098,0.781585,12.540622,1.994714
4,0.385913,-0.922912,9.923636,13.898736,1.344335


### 使用k均值构建模型

In [26]:
# 导入算法
from sklearn.cluster import KMeans
# 确定聚类中心数量
k = 5
# 构建模型, 训练模型
kmeans_model = KMeans(n_clusters=k, random_state=123).fit(data_feature_scaled)
kmeans_model

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=123, tol=0.0001, verbose=0)

In [27]:
# 得到每个样本数据的标签
kmeans_model.labels_

array([1, 1, 1, ..., 4, 2, 2], dtype=int32)

In [29]:
# 统计不同类别样本数目
r1 = pd.Series(kmeans_model.labels_).value_counts()
r1

4    24638
3    15735
2    12119
1     5337
0     4215
dtype: int64

In [28]:
# 得到每类样本的聚类中心
kmeans_model.cluster_centers_

array([[ 4.65625727e-02, -1.99113055e-03, -2.30142183e-01,
        -2.34464735e-01,  2.17865520e+00],
       [ 4.83657973e-01, -7.99400212e-01,  2.48317490e+00,
         2.42445945e+00,  3.09237966e-01],
       [-3.13368048e-01,  1.68669161e+00, -5.73935737e-01,
        -5.36782673e-01, -1.74608430e-01],
       [ 1.16084862e+00, -3.77377217e-01, -8.66405041e-02,
        -9.45551300e-02, -1.56599649e-01],
       [-7.00313344e-01, -4.15035589e-01, -1.60898126e-01,
        -1.60646112e-01, -2.56723170e-01]])

In [30]:
csv_data = pd.DataFrame(kmeans_model.cluster_centers_)
csv_data.to_csv('csv_data.csv')